## Imports

In [117]:
import pandas as pd
import torch

In [5]:
from transformers import DistilBertTokenizerFast, DistilBertModel

In [1]:
%reload_ext autoreload
%autoreload 2

## Read data

In [2]:
#from read_dataset import read_dataset

In [3]:
#dataset = read_dataset(path='SQUAD MATERIAL/training_set.json', validation_set_perc=20)

In [5]:
#train_df = pd.DataFrame(dataset[0], columns=['id', 'title', 'context', 'question', 'start', 'end'])
#train_df.to_csv('./data/train_df.csv')

In [6]:
#val_df = pd.DataFrame(dataset[1], columns=['id', 'title', 'context', 'question', 'start', 'end'])
#val_df.to_csv('./data/val_df.csv')

In [3]:
train_df = pd.read_csv('./data/train_df.csv')
val_df = pd.read_csv('./data/val_df.csv')

### Read questions and contexts

In [4]:
questions = list(train_df['question'].values)
contexts = list(train_df['context'].values)

## Embedding

In [28]:
# Load tokenizer and transformers

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased-distilled-squad')
model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')

Some weights of the model checkpoint at distilbert-base-cased-distilled-squad were not used when initializing DistilBertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# Tokenize questions and contexts
max_length = model.config.max_position_embeddings
doc_stride = 128

#tokenized = tokenizer(
#    questions[:10],
#    contexts[:10],
#    max_length=max_length,
#    truncation="only_second",
#    return_overflowing_tokens=True,
#    return_offsets_mapping=True,
#    stride=doc_stride,
#    return_attention_mask=True
#)

In [112]:
tokenizer.decode(tokenized['input_ids'][0])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [18]:
tokenized['offset_mapping'][0]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 11),
 (12, 15),
 (16, 22),
 (23, 27),
 (28, 37),
 (38, 44),
 (45, 47),
 (48, 52),
 (53, 55),
 (56, 59),
 (59, 63),
 (64, 70),
 (70, 71),
 (0, 0),
 (0, 13),
 (13, 15),
 (15, 16),
 (17, 20),
 (21, 27),
 (28, 31),
 (32, 33),
 (34, 42),
 (43, 52),
 (52, 53),
 (54, 56),
 (56, 58),
 (59, 62),
 (63, 67),
 (68, 76),
 (76, 77),
 (77, 78),
 (79, 83),
 (84, 88),
 (89, 91),
 (92, 93),
 (94, 100),
 (101, 107),
 (108, 110),
 (111, 114),
 (115, 121),
 (122, 126),
 (126, 127),
 (128, 139),
 (140, 142),
 (143, 148),
 (149, 151),
 (152, 155),
 (156, 160),
 (161, 169),
 (170, 173),
 (174, 180),
 (181, 183),
 (183, 184),
 (185, 187),
 (188, 189),
 (190, 196),
 (197, 203),
 (204, 206),
 (207, 213),
 (214, 218),
 (219, 223),
 (224, 226),
 (226, 229),
 (229, 232),
 (233, 237),
 (238, 241),
 (242, 248),
 (249, 250),
 (250, 251),
 (251, 254),
 (254, 256),
 (257, 259),
 (260, 262),
 (263, 264),
 (264, 265),
 (265, 268),
 (268, 269),
 (269, 270),
 (271, 275),
 (276, 278),
 (279, 2

`tokenized['offset_mapping'][0]` returna le tuple (start,end) di ogni parola dell'input (query, context)
Problema: risposte si accavallano attorno a max_length
Soluzione: riscalare tuple di contesti tagliati (invece che 0, allinearli alla risposta)
[soluzione](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb#scrollTo=iLekL6Un9D70&line=24&uniqifier=1)

## TODO:

- creare dataframe con contesti splittati (lunghezza 512, overlapping 256)
    - riscalare answer_start e answer_end per ogni contesto
    - lo split che contiene la risposta mantiene answer_start e answer_end, gli altri split dello stesso contesto vanno trattati (assegniamo (0,0)? oppure scartiamo)

In [157]:
tokenized = tokenizer(
    questions[:10],
    contexts[:10],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride,
    return_attention_mask=True,
    padding='max_length'
)

In [78]:
train_df['question'].apply(lambda x: len(x.strip().split(' '))).max()

40

In [119]:
device = torch.device('cpu')

In [158]:
bert_dict = {}

bert_dict['input_ids'] = torch.IntTensor(tokenized['input_ids']).to(device)
bert_dict['attention_mask'] = torch.IntTensor(tokenized['attention_mask']).to(device)

In [159]:
transformed = model(**bert_dict)

In [160]:
dropped = torch.nn.Dropout(0.3)(transformed[0])

In [161]:
logits = torch.nn.Linear(768, 2)(dropped)

In [162]:
start_logits, end_logits = logits.split(1, dim=-1)

In [163]:
start_logits = start_logits.squeeze(-1)
end_logits = end_logits.squeeze(-1)

In [164]:
outputs = (start_logits, end_logits)

## Network

In [ ]:
class QA(torch.nn.Module):

    def __init__(self, hidden_size=768, num_labels=2, dropout_rate=0.5):
        super(QA, self).__init__()
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased-distilled-squad')
        self.model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')
        self.dropout = torch.nn.Dropout(dropout_rate)
        #self.extra_linear = torch.nn.Linear(self.hidden_size, self.hidden_size)
        #self.extra_linear_tanh = torch.nn.Tanh()
        self.dense = torch.nn.Linear(self.hidden_size, self.num_labels)

    def forward(self, inputs):
        # Unpack inputs
        questions, contexts = inputs
        # Tokenizer
        tokenized = self.tokenizer(
            questions,
            contexts,
            max_length=max_length,
            truncation="only_second",
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            stride=doc_stride,
            return_attention_mask=True,
            padding='max_length'
        )
        # Put to device
        bert_dict = {}

        bert_dict['input_ids'] = torch.IntTensor(tokenized['input_ids']).to(self.device)
        bert_dict['attention_mask'] = torch.IntTensor(tokenized['attention_mask']).to(self.device)
        # Transformers 
        transformed = self.model(**bert_dict)
        # Dropout
        dropped = self.dropout(transformed[0])
        # Obtain logits
        logits = self.dense(dropped) #(None, seq_len, hidden_size)*(hidden_size, 2)=(None, seq_len, 2)
        start_logits, end_logits = logits.split(1, dim=-1)    #(None, seq_len, 1), (None, seq_len, 1)
        start_logits = start_logits.squeeze(-1)  #(None, seq_len)
        end_logits = end_logits.squeeze(-1)    #(None, seq_len)
        # --- 4) Prepare output tuple
        outputs = (start_logits, end_logits)
        
        return outputs